Criaremos uma função para ler o arquivo de *annotations* em HTML criado no Web CVAT.


In [42]:
import xml.etree.ElementTree as ET

# Função para ler e processar o arquivo XML
# Retorna um dicionário onde a chave é o nome do arquivo,
# e o valor é a label e as coordenadas do bounding box
# Ex: bounding_boxes['0-01-V1-B.png'] = {('apple', (448.50 284.60 637.25 485.31))}
# No caso, as coordenadas são xtl (x top left), ytl, xbr e ybr, nessa ordem
def get_annotations_from_xml(xml_file) -> dict:
    image_annotation = {}
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Iterar sobre cada imagem
    for image in root.findall('image'):
        filename = image.get('name')
        fruit_type = image[0].get('label')
        xtl = float(image[0].get('xtl'))
        ytl = float(image[0].get('ytl'))
        xbr = float(image[0].get('xbr'))
        ybr = float(image[0].get('ybr'))

        # Adicionar ao dicionário com o nome do arquivo como chave
        image_annotation[filename] = {(fruit_type, (xtl, ytl, xbr, ybr))}

    return image_annotation


Criaremos uma função para ler o arquivo de *annotations* fruits-DL-1024x768-annotations.tar.gz e usar a função criada acima.

In [43]:
import gdown
import tarfile

# URL para baixar o arquivo do Google Drive via gdown (use apenas o ID do arquivo)
file_id = '1n3oumxGZu-Lhm2V8_VLpHyJdo2yvUvsI'
download_url = f"https://drive.google.com/uc?id={file_id}"

# Baixar o arquivo .tar.gz
annotations_targz_filename = 'annotations.tar.gz'
gdown.download(download_url, annotations_targz_filename, quiet=True)

annotations = {}
with tarfile.open(annotations_targz_filename, 'r:gz') as tar:
    # Iterar sobre os arquivos no tar
    for member in tar.getmembers():
        if 'annotations.xml' in member.name:
        # Extrair o arquivo xml
            file = tar.extractfile(member)
            if file:
                annotations = get_annotations_from_xml(file)

Daqui em diante, a intenção era utilizar o bounding box para fazer uma média da cor dentro da caixa (seria melhor usar o ground truth para isso, mas ele não deu certo para algumas imagens). Além disso, seria usado *width* e *height* de cada bounding box. Com essas três características, seria treinado algum tipo de classificador.

O dicionário com as *bounding boxes* é:

In [45]:
print(annotations)

{'0-01-V1-B.png': {('apple', (448.5, 284.6, 637.25, 485.31))}, '0-01-V1-W.png': {('apple', (426.4, 297.5, 659.35, 520.3))}, '0-01-V2-B.png': {('apple', (453.78, 258.12, 664.72, 490.53))}, '0-01-V2-W.png': {('apple', (454.15, 286.62, 662.5, 508.29))}, '0-02-V1-B.png': {('apple', (415.66, 283.29, 632.53, 496.82))}, '0-02-V1-W.png': {('apple', (480.8, 263.68, 687.67, 470.18))}, '0-02-V2-B.png': {('apple', (431.21, 268.86, 633.64, 491.64))}, '0-02-V2-W.png': {('apple', (474.14, 281.44, 675.83, 484.24))}, '0-03-V1-B.png': {('apple', (448.97, 258.49, 688.04, 509.4))}, '0-03-V1-W.png': {('apple', (459.33, 311.04, 687.67, 518.66))}, '0-03-V2-B.png': {('apple', (424.1, 308.19, 672.79, 547.11))}, '0-03-V2-W.png': {('apple', (451.19, 277.37, 645.26, 492.76))}, '0-04-V1-B.png': {('apple', (425.88, 253.84, 659.02, 492.31))}, '0-04-V1-W.png': {('apple', (416.11, 267.16, 645.7, 486.98))}, '0-04-V2-B.png': {('apple', (442.31, 292.47, 667.46, 521.18))}, '0-04-V2-W.png': {('apple', (424.1, 288.03, 632.8